In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%pip install wandb -qU
%pip install --no-deps scikeras
%pip install tensorflow-addons
%pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 11.1 MB/s eta 0:00:00


In [108]:
# import sys
# sys.path.append('/content/drive/MyDrive/Lab Work/Zach/Zach/NNS')

%cd /content/drive/MyDrive/Lab Work/Zach/Zach
import NN5Hl, NN4Hl, NN3Hl, NN2Hl, NN1Hl, init


/content/drive/MyDrive/Lab Work/Zach/Zach


In [109]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
import wandb
from wandb.keras import WandbCallback
import json
from pathlib import Path

In [110]:
class DataProcesses:
    def __init__(self, RealData, SimData):
        self.RealData = pd.read_csv(RealData)
        self.SimData = pd.read_csv(SimData)
        self.CombinedData = None
        self.EncodedData = None

    def Data_merge(self):

        #Declare labels for simulated data and real data
        self.SimData['type'] = 0 #SimData
        self.RealData['type'] = 1 #RealData

        #Combine the real and simulated data into on dataset
        self.CombinedData = pd.concat([self.SimData, self.RealData], axis=0)


    def Data_Encode(self):
        #init Encode from sklearn.preprocessing
        label_encoder = LabelEncoder()

        #Make a copy of Combineddata for encoding
        self.EncodedData = self.CombinedData.copy()


        # Encode the col Topcis, msg and nodes
        self.EncodedData['topics']= label_encoder.fit_transform(self.EncodedData['topics'])
        self.EncodedData['msg']= label_encoder.fit_transform(self.EncodedData['msg'])
        self.EncodedData['node']= label_encoder.fit_transform(self.EncodedData['node'])

    def process(self):
        #Run both Data merge and encoding function
        self.Data_merge()
        self.Data_Encode()


In [111]:
class func:
    def __init__(self, NumHL, dataset):
        self.dataset = dataset
        self.NumHL = NumHL

    def data_split(self):
        predict = 'type'
        X = self.dataset.drop(predict, axis=1)
        y = self.dataset[predict]
        X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(X, y, test_size=0.3, shuffle=True)
        return X_train, X_test, y_train, y_test

    def model_selection(self):
        if NumHL == 1:
            nn = NN1Hl.NeuralNet(wandb.config.fc_layer_size)
            model = nn.create_model()
            return model
        elif NumHL == 2:
            nn = NN2Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 3:
            nn = NN3Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 4:
            nn = NN4Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.Frth_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model
        elif NumHL == 5:
            nn = NN5Hl.NeuralNet(wandb.config.fc_layer_size,wandb.config.sc_layer_size,wandb.config.Td_layer_size,wandb.config.Frth_layer_size,wandb.config.Fif_layer_size,wandb.config.dropout)
            model = nn.create_model()
            return model

    def get_optimizer(self, lr=1e-3, optimizer="adam"):
        "Select optmizer between adam and sgd with momentum"
        if optimizer.lower() == "adam":
            return tf.keras.optimizers.Adam(learning_rate=lr)
        if optimizer.lower() == "sgd":
            return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
        if optimizer.lower() == "rmsprop":
            return tf.keras.optimizers.RMSprop(learning_rate=lr)
        if optimizer.lower() == "nadam":
            return tf.keras.optimizers.Nadam(learning_rate=lr)
        if optimizer.lower() == "adadelta":
            return tf.keras.optimizers.Adadelta(learning_rate=lr)
        if optimizer.lower() == "adagrad":
            return tf.keras.optimizers.Adagrad(learning_rate=lr)
        if optimizer.lower() == "adamax":
            return tf.keras.optimizers.Adamax(learning_rate=lr)
        if optimizer.lower() == "ftrl":
            return tf.keras.optimizers.Ftrl(learning_rate=lr)
        if optimizer.lower() == "adamw":
            return tf.keras.optimizers.AdamW(learning_rate=lr)

In [112]:
RealData_Train = "/content/drive/MyDrive/Lab Work/Zach/Zach/TrainingData/RWB_RealDataCombined.csv"
SimData_Train = "/content/drive/MyDrive/Lab Work/Zach/Zach/TrainingData/RWB_SimDataCombined.csv"

NewData  = DataProcesses(RealData_Train,SimData_Train)
NewData.process()

In [113]:
proj_name = 'NN_Turing'
entity_name="dt_zach"
NumHL = 1

# Create an instance of Wandb_sweep
functions = func(NumHL, NewData.EncodedData)

# Call the run method on the instance
X_train, X_test, y_train, y_test = functions.data_split()


In [139]:
sweep_config = json.load(Path(f'Configs/{NumHL}HLSweepConfig.json').open('r'))
sweep_id = wandb.sweep(sweep=sweep_config, project=proj_name, entity=entity_name)

Create sweep with ID: e667d46e
Sweep URL: https://wandb.ai/dt_zach/NN_Turing/sweeps/e667d46e


In [140]:
import pprint
pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'f1_score'},
 'name': 'sweep',
 'parameters': {'batch_size': {'values': [10, 15, 25, 32, 50, 64, 75, 100]},
                'epochs': {'values': [50, 75, 100, 125, 150, 200, 250, 300]},
                'fc_layer_size': {'values': [7, 9, 11, 13, 15, 20, 25, 30, 50]},
                'lr': {'max': 0.1, 'min': 0.0001},
                'optimizer': {'values': ['adam',
                                         'SGD',
                                         'RMSprop',
                                         'nadam',
                                         'Adadelta',
                                         'Adagrad',
                                         'Adamax',
                                         'Ftrl',
                                         'AdamW']}}}


In [141]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TestLabfab"
        wandb.config.dataset_name = "Labfab"

        # initialize model
        model = functions.model_selection()
        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer)

In [142]:
def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(optimizer=functions.get_optimizer(lr, optimizer) ,
               loss="binary_crossentropy",
               metrics=["binary_accuracy",
               tfa.metrics.F1Score(num_classes=2,
                                  average = 'micro',
                                  threshold=0.5)
                        ])

    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[WandbCallback()])

In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=150)

wandb: Agent Starting Run: 4zmu6i2u with config:
wandb: 	batch_size: 10
wandb: 	epochs: 50
wandb: 	fc_layer_size: 11
wandb: 	lr: 0.05672603447868575
wandb: 	optimizer: AdamW


Epoch 1/50
 1/16 [>.............................] - ETA: 10s - loss: 1.1553 - binary_accuracy: 0.3000 - f1_score: 0.3636

wandb: Adding directory to artifact (/content/wandb/run-20230801_165400-4zmu6i2u/files/model-best)... Done. 0.0s


16/16 [==============================] - 2s 64ms/step - loss: 4.2059 - binary_accuracy: 0.4906 - f1_score: 0.5091 - val_loss: 0.6840 - val_binary_accuracy: 0.5797 - val_f1_score: 0.0000e+00
Epoch 2/50
 1/16 [>.............................] - ETA: 0s - loss: 0.8222 - binary_accuracy: 0.4000 - f1_score: 0.0000e+00

wandb: Adding directory to artifact (/content/wandb/run-20230801_165400-4zmu6i2u/files/model-best)... Done. 0.0s


...

7/7 [==============================] - 1s 126ms/step - loss: 7.7911 - binary_accuracy: 0.4654 - f1_score: 0.0000e+00 - val_loss: 5.3256 - val_binary_accuracy: 0.5797 - val_f1_score: 0.0000e+00
Epoch 278/300
1/7 [===>..........................] - ETA: 0s - loss: 8.8373 - binary_accuracy: 0.4000 - f1_score: 0.0000e+00

wandb: Adding directory to artifact (/content/wandb/run-20230801_165604-dgj79f2w/files/model-best)... Done. 0.0s


7/7 [==============================] - 1s 170ms/step - loss: 7.6684 - binary_accuracy: 0.4654 - f1_score: 0.0000e+00 - val_loss: 5.2403 - val_binary_accuracy: 0.5797 - val_f1_score: 0.0000e+00
Epoch 279/300
1/7 [===>..........................] - ETA: 0s - loss: 5.9116 - binary_accuracy: 0.4400 - f1_score: 0.0000e+00

wandb: Adding directory to artifact (/content/wandb/run-20230801_165604-dgj79f2w/files/model-best)... Done. 0.0s


7/7 [==============================] - 1s 178ms/step - loss: 7.5410 - binary_accuracy: 0.4654 - f1_score: 0.0000e+00 - val_loss: 5.1476 - val_binary_accuracy: 0.5797 - val_f1_score: 0.0000e+00
Epoch 280/300


In [56]:
# Fetch sweep runs
sweep_runs = wandb.Api().sweep(f"{entity_name}/{proj_name}/{sweep_id}")


In [ ]:
import wandb

# Fetch the sweep
sweep = wandb.Api().sweep(f"{entity_name}/{proj_name}/{sweep_id}")

# Define the metric name you want to optimize
metric_name = "f1_score"

# Initialize variables to track the best run and best metric value
best_run = None
best_metric_value = float('-inf')  # For maximization, use float('-inf'), for minimization, use float('inf')

# Get a list of all runs from the sweep
all_runs = list(sweep.runs)

# Iterate over all runs to find the best run
for run in all_runs:
    run_summary = run.summary
    if metric_name in run_summary:
        metric_value = run_summary[metric_name]
        if metric_value > best_metric_value:  # For minimization, use metric_value < best_metric_value
            best_metric_value = metric_value
            best_run = run

if best_run is not None:
    best_config = best_run.config
    print("Best Configuration:")
    print(best_config)
    print(f"Best {metric_name}:", best_metric_value)
else:
    print(f"No runs found with the specified metric ({metric_name}).")


Best Configuration:
{'lr': 0.009204918186867911, 'epochs': 200, 'dropout': 0, 'optimizer': 'AdamW', 'batch_size': 25, 'dataset_name': 'Labfab', 'fc_layer_size': 50, 'sc_layer_size': 50, 'architecture_name': 'TestLabfab'}
Best f1_score: 0.7326732873916626
